# Geodes with EODAG

In this tutorial we will see how to use [EODAG](https://github.com/CS-SI/eodag) to search and download data on the [Geodes portal](https://geodes.cnes.fr).

*Disable some warnings related to geodes SSL verification disabled*

In [1]:
import requests
requests.packages.urllib3.disable_warnings() 

## 1. Credentials configuration

As explained on [Provider registration - geodes](https://eodag.readthedocs.io/en/stable/getting_started_guide/register.html#geodes), go to https://geodes-portal.cnes.fr, then login or create an account by clicking on `Log in` in the top-right corner. 

Once logged-in, create an API key in the user settings page, and use it as `apikey` in EODAG provider auth credentials.

- Update your EODAG user configuration file:
```yml
geodes:
    auth:
        credentials:
            apikey: your-api-key
```
- Or set it using environment variables:

In [2]:
import os
from getpass import getpass

os.environ["EODAG__GEODES__AUTH__CREDENTIALS__APIKEY"] = getpass("Input apikey: ")

Input apikey:  ········


## 2. Search

Let's instantiate EODAG `EODataAccessGateway`

In [3]:
from eodag import EODataAccessGateway

dag = EODataAccessGateway()

Then look for available product types on `geodes`

In [4]:
product_types = dag.list_product_types("geodes")
[pt["ID"] for pt in product_types]

['MUSCATE_LANDSAT_LANDSAT8_L2A',
 'MUSCATE_Landsat57_LANDSAT5_N2A',
 'MUSCATE_Landsat57_LANDSAT7_N2A',
 'MUSCATE_OSO_RASTER_L3B-OSO',
 'MUSCATE_OSO_VECTOR_L3B-OSO',
 'MUSCATE_PLEIADES_PLEIADES_ORTHO',
 'MUSCATE_PLEIADES_PLEIADES_PRIMARY',
 'MUSCATE_SENTINEL2_SENTINEL2_L3A',
 'MUSCATE_SPOTWORLDHERITAGE_SPOT1_L1C',
 'MUSCATE_SPOTWORLDHERITAGE_SPOT2_L1C',
 'MUSCATE_SPOTWORLDHERITAGE_SPOT3_L1C',
 'MUSCATE_SPOTWORLDHERITAGE_SPOT4_L1C',
 'MUSCATE_SPOTWORLDHERITAGE_SPOT5_L1C',
 'MUSCATE_Spirit_SPOT5_L1A',
 'MUSCATE_VENUSVM05_VM5_L1C',
 'MUSCATE_VENUSVM05_VM5_L2A',
 'MUSCATE_VENUSVM05_VM5_L3A',
 'MUSCATE_VENUS_VM1_L1C',
 'MUSCATE_VENUS_VM1_L2A',
 'MUSCATE_VENUS_VM1_L3A',
 'MUSCATE_WaterQual_SENTINEL2_L2B-WATER',
 'PEPS_S2_L2A',
 'PEPS_S3_L1',
 'POSTEL_LANDCOVER_GLOBCOVER',
 'POSTEL_RADIATION_BRDF',
 'POSTEL_RADIATION_DLR',
 'POSTEL_RADIATION_SURFACEALBEDO',
 'POSTEL_RADIATION_SURFACEREFLECTANCE',
 'POSTEL_VEGETATION_FAPAR',
 'POSTEL_VEGETATION_FCOVER',
 'POSTEL_VEGETATION_LAI',
 'POSTEL_VEGETA

Search for October-2024 Sentinel 2 L1C data over South-West of France with less that 5% of cloud coverage

In [5]:
search_bbox = [1, 43, 2, 44]
search_criteria = dict(
    provider="geodes",
    productType="S2_MSI_L1C",
    start="2024-10-01",
    end="2024-10-30",
    geom=search_bbox,
    cloudCover=5,
    count=True,
)

results = dag.search_all(**search_criteria)
results

SearchResult([EOProduct(id=S2B_MSIL1C_20241008T103829_N0511_R008_T31TCJ_20241008T131242, provider=geodes),
              EOProduct(id=S2A_MSIL1C_20241013T104001_N0511_R008_T31TCH_20241013T124120, provider=geodes)])

View found products extents and search area on a map

In [7]:
import folium

# Create a map zoomed over the search area
fmap = folium.Map([43.5, 1.5], zoom_start=8)

# Create a layer that represents the search area in red
folium.Rectangle(
    bounds=[search_bbox[::-1][2:4], search_bbox[::-1][0:2]], 
    color="red", 
    tooltip="Search extent"
).add_to(fmap)

# Create a layer that maps the products found
folium.GeoJson(
    data=results, tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)

fmap

## 3. Download products to `/work/scratch/data/{USER}"` directory

In [8]:
paths = dag.download_all(results, output_dir=f"/work/scratch/data/{os.environ['JUPYTERHUB_USER']}")
paths

Downloaded products:   0%|          | 0/2 [00:00<?, ?product/s]

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

['/work/scratch/data/brunats/S2B_MSIL1C_20241008T103829_N0511_R008_T31TCJ_20241008T131242',
 '/work/scratch/data/brunats/S2A_MSIL1C_20241013T104001_N0511_R008_T31TCH_20241013T124120']

In [9]:
!tree {paths[0]}

/work/scratch/data/brunats/S2B_MSIL1C_20241008T103829_N0511_R008_T31TCJ_20241008T131242
├── DATASTRIP
│   └── DS_2BPS_20241008T131242_S20241008T104519
│       ├── MTD_DS.xml
│       └── QI_DATA
│           ├── FORMAT_CORRECTNESS.xml
│           ├── GENERAL_QUALITY.xml
│           ├── GEOMETRIC_QUALITY.xml
│           ├── RADIOMETRIC_QUALITY.xml
│           └── SENSOR_QUALITY.xml
├── GRANULE
│   └── L1C_T31TCJ_A039645_20241008T104519
│       ├── AUX_DATA
│       │   ├── AUX_CAMSFO
│       │   └── AUX_ECMWFT
│       ├── IMG_DATA
│       │   ├── T31TCJ_20241008T103829_B01.jp2
│       │   ├── T31TCJ_20241008T103829_B02.jp2
│       │   ├── T31TCJ_20241008T103829_B03.jp2
│       │   ├── T31TCJ_20241008T103829_B04.jp2
│       │   ├── T31TCJ_20241008T103829_B05.jp2
│       │   ├── T31TCJ_20241008T103829_B06.jp2
│       │   ├── T31TCJ_20241008T103829_B07.jp2
│       │   ├── T31TCJ_20241008T103829_B08.jp2
│       │   ├── T31TCJ_20241008T103829_B09.jp2
│       │   ├── T31TCJ_20241008T103829_B10.j